### Read all Comments and all References from Docx [Updatede, Preferred]

[Cross Platform Solution]

In [1]:
from docx import Document
from lxml import etree
import zipfile
import pandas as pd

ooXMLns = {'w':'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

In [2]:
BASE = ''
fname = 'testdoc.docx'

### 1.1 Get Comment and Reference Paragraph

In [3]:
#Function to extract all the comments of document(Same as accepted answer)
#Returns a dictionary with comment id as key and comment string as value
def get_document_comments(docxFileName):
    comments_dict={}
    docxZip = zipfile.ZipFile(docxFileName)
    commentsXML = docxZip.read('word/comments.xml')
    et = etree.XML(commentsXML)
    comments = et.xpath('//w:comment',namespaces=ooXMLns)
    for c in comments:
        comment=c.xpath('string(.)',namespaces=ooXMLns)
        comment_id=c.xpath('@w:id',namespaces=ooXMLns)[0]
        comments_dict[comment_id]=comment
    docxZip.close()
    return comments_dict

#Function to fetch all the comments in a paragraph
def paragraph_comments(paragraph,comments_dict):
    comments=[]
    for run in paragraph.runs:
        comment_reference=run._r.xpath("./w:commentReference")
        if comment_reference:
            comment_id=comment_reference[0].xpath('@w:id',namespaces=ooXMLns)[0]
            comment=comments_dict[comment_id]
            comments.append(comment)
    return comments

#Function to fetch "all comments" with their referenced "all paragraphs"
#This will return list like this [{'Paragraph text': [comment 1,comment 2]}]
def comments_with_reference_paragraph(docxFileName):
    document = Document(docxFileName)
    comments_dict=get_document_comments(docxFileName)
    comments_with_their_reference_paragraph=[]
    cflag = False
    ref_parts = []
    for paragraph in document.paragraphs:
        
        if 'w:commentRangeStart' in paragraph._p.xml: # start getting all referred paragraphs
            cflag = True
        if cflag:
            ref_parts.append(paragraph.text) # continue getting all referred paragraphs
            
        if 'w:commentRangeEnd' in paragraph._p.xml: # end getting all referred paragraphs
            cflag = False 
            
            if comments_dict: 
                comments=paragraph_comments(paragraph,comments_dict) # get all comments for this block of referred objects
                if comments:
                    comments_with_their_reference_paragraph.append({'ref objects':'|||'.join(ref_parts), 'comments':'|||'.join(comments)}) # dataframe friendly format
            
    return comments_with_their_reference_paragraph

annot_list = comments_with_reference_paragraph(BASE+fname)

In [4]:
pd.DataFrame(annot_list)

,ref objects,comments
0,Video provides a powerful way to help you prov...,dor comment one
1,Video provides a powerful way to help you prov...,dor comment two|||dor comment three


Reference:

https://stackoverflow.com/questions/47390928/extract-docx-comments
